In [65]:
import utils.conf as conf
import preprocessing.data_cleaner as cleaner
import preprocessing.feature_extractor as extractor
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
train, test = cleaner.clean()

In [76]:
train

,TripType,VisitNumber,ScanCount,FinelineNumber,Encoded_DepartmentDescription,Encoded_Upc,Encoded_Weekday
0,999,5,-1,1000,21,66661,5
1,30,7,1,8931,63,58982,5
2,30,7,1,4504,51,41387,5
3,26,8,2,3565,50,9252,5
4,26,8,2,1017,50,7860,5
5,26,8,2,1017,50,7871,5
6,26,8,1,1017,50,7859,5
7,26,8,1,2802,50,35057,5
8,26,8,1,4501,50,9272,5
9,26,8,-1,3565,50,9245,5


**Cross validation**

In [67]:
y = train['TripType']
X = train.drop(['TripType'], axis=1)

In [68]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [69]:
vn = train['VisitNumber'].unique()

In [70]:
from sklearn.model_selection import train_test_split
train_vn, test_vn = train_test_split(vn, test_size=0.4)
train_index = train['VisitNumber'].isin(train_vn)
y_train = train[train_index]['TripType']
X_train = train[train_index].drop('TripType', axis=1)
y_test = train[-train_index]['TripType']
X_test = train[-train_index].drop('TripType', axis=1)

In [71]:
X_train = extractor.aggreate_features(X_train)
X_test = extractor.aggreate_features(X_test)

** add feature **
* number of returned items per trip

In [72]:
X_train.drop('VisitNumber',axis=1, inplace=True)

In [73]:
X_test.drop('VisitNumber', axis=1, inplace=True)

In [74]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, n_jobs=4)
clf.fit(X_train,y_train)
clf_probs = clf.predict_proba(X_test)

In [75]:
import numpy as np
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d : %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]],importances[indices[f]]))
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(clf, X_test, y_test.values)
scores = clf.score(X_test,y_test)
print(scores)

from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, clf_probs)
print(logloss_score)

Feature ranking:
1. feature 11 : Encoded_DepartmentDescription_has_max_by_vn (0.135551)
2. feature 13 : ScanCount_purchased_items_by_vn (0.129917)
3. feature 9 : FinelineNumber_has_max_by_vn (0.124469)
4. feature 7 : Encoded_Upc_nunique_groupby_vn (0.096143)
5. feature 6 : FinelineNumber_nunique_groupby_vn (0.080364)
6. feature 10 : Encoded_DepartmentDescription_max_by_vn (0.066343)
7. feature 5 : Encoded_DepartmentDescription_nunique_groupby_vn (0.065772)
8. feature 4 : Encoded_Weekday (0.064907)
9. feature 3 : Encoded_Upc (0.060493)
10. feature 2 : Encoded_DepartmentDescription (0.057254)
11. feature 1 : FinelineNumber (0.053116)
12. feature 8 : FinelineNumber_max_by_vn (0.032754)
13. feature 12 : ScanCount_returned_items_by_vn (0.021712)
14. feature 0 : ScanCount (0.011204)
0.608704168123
2.9213188378


Feature ranking:
1. feature 11 : Encoded_DepartmentDescription_has_max_by_vn (0.135551)
2. feature 13 : ScanCount_purchased_items_by_vn (0.129917)
3. feature 9 : FinelineNumber_has_max_by_vn (0.124469)
4. feature 7 : Encoded_Upc_nunique_groupby_vn (0.096143)
5. feature 6 : FinelineNumber_nunique_groupby_vn (0.080364)
6. feature 10 : Encoded_DepartmentDescription_max_by_vn (0.066343)
7. feature 5 : Encoded_DepartmentDescription_nunique_groupby_vn (0.065772)
8. feature 4 : Encoded_Weekday (0.064907)
9. feature 3 : Encoded_Upc (0.060493)
10. feature 2 : Encoded_DepartmentDescription (0.057254)
11. feature 1 : FinelineNumber (0.053116)
12. feature 8 : FinelineNumber_max_by_vn (0.032754)
13. feature 12 : ScanCount_returned_items_by_vn (0.021712)
14. feature 0 : ScanCount (0.011204)

0.608704168123

2.9213188378

Feature ranking:
1. feature 12 : Encoded_DepartmentDescription_has_max_by_vn (0.134543)
2. feature 5 : ScanCount_sum_groupby_vn (0.128942)
3. feature 10 : FinelineNumber_has_max_by_vn (0.123942)
4. feature 8 : Encoded_Upc_nunique_groupby_vn (0.098007)
5. feature 7 : FinelineNumber_nunique_groupby_vn (0.092697)
6. feature 6 : Encoded_DepartmentDescription_nunique_groupby_vn (0.067066)
7. feature 4 : Encoded_Weekday (0.063992)
8. feature 11 : Encoded_DepartmentDescription_max_by_vn (0.060958)
9. feature 3 : Encoded_Upc (0.060370)
10. feature 2 : Encoded_DepartmentDescription (0.057118)
11. feature 1 : FinelineNumber (0.053392)
12. feature 9 : FinelineNumber_max_by_vn (0.031842)
13. feature 13 : ScanCount_returned_items_by_vn (0.017312)
14. feature 0 : ScanCount (0.009820)

0.610022798277

2.906785231


Feature ranking:
1. feature 5 : ScanCount_sum_groupby_vn (0.146234)
2. feature 12 : Encoded_DepartmentDescription_has_max_by_vn (0.136101)
3. feature 10 : FinelineNumber_has_max_by_vn (0.126257)
4. feature 8 : Encoded_Upc_nunique_groupby_vn (0.093432)
5. feature 7 : FinelineNumber_nunique_groupby_vn (0.086363)
6. feature 6 : Encoded_DepartmentDescription_nunique_groupby_vn (0.068791)
7. feature 11 : Encoded_DepartmentDescription_max_by_vn (0.065487)
8. feature 4 : Encoded_Weekday (0.065123)
9. feature 3 : Encoded_Upc (0.060467)
10. feature 2 : Encoded_DepartmentDescription (0.055747)
11. feature 1 : FinelineNumber (0.052079)
12. feature 9 : FinelineNumber_max_by_vn (0.032862)
13. feature 0 : ScanCount (0.011058)
    
0.6123093149686295

2.64528643116

**Prepare for submission**

In [3]:
train = extractor.aggreate_features(train)
test = extractor.aggreate_features(test)

In [4]:
import models.randomforest as randomforest

y_train = train['TripType']
X_train = train.drop(['TripType'],axis=1)
rf_clf = randomforest.RandomForest()
rf_clf.fit(X_train, y_train)

RandomForest(n_estimators=50, n_jobs=4)

In [5]:
preds, classes = rf_clf.predict_proba(test)

In [6]:
column_names = ['VisitNumber']
for cls in classes:
    column_names.append('TripType_' + str(cls))

In [12]:
import pandas as pd
out = pd.DataFrame(columns=column_names)
out['VisitNumber'] = test['VisitNumber']
out.ix[:,'TripType_3':] = preds
out = out.groupby('VisitNumber').mean()
out.to_csv('submission.csv', sep=',')

In [14]:
out.shape

(95674, 38)